# Implimenting abstractive text summerization

In [9]:
#import all the necessary libraries
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [71]:
#generate precize sentences
def read_article(file_form,num):
    article=[]
    if num==1:
        file = open(file_form, "r")
        filedata = file.readlines()
        for i in range(len(filedata)):
            article.append(filedata[i].split(". "))
    if num==2:
        text_url=text_article(file_form)
        article.append(text_url.split("."))
    if num==3:
        article.append(file_form.split("."))
    article_text = ""
    #print(len(article[0]))    
    sentences = []
    for sentence in article[0]:
        #print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" ")) 
    return sentences

In [83]:
#import libraries
import bs4 as bs  
import urllib.request  
import re
#processing data from url
def text_article(url):
    scraped_data = urllib.request.urlopen(url)  
    article = scraped_data.read()
    parsed_article = bs.BeautifulSoup(article,'lxml')
    paragraphs = parsed_article.find_all('p')
    article_text = ""
    for p in paragraphs:  
        article_text += p.text
    # Removing Square Brackets and Extra Spaces
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)  
    article_text = re.sub(r'\s+', ' ', article_text)
    print(article_text)
    # Removing special characters and digits
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )  
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text) 
    return formatted_article_text

In [84]:
#building cosine difference between two sentence vector
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
    all_words = list(set(sent1 + sent2))
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [85]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)
    return similarity_matrix

In [86]:
def generate_summary(file_name, num):
    stop_words = stopwords.words('english')
    summarize_text = []
    # Step 1 - Read text and tokenize
    sentences =  read_article(file_name,num)
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)
    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)
    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True) 
    for i in range(len(ranked_sentence)):
        summarize_text.append(" ".join(ranked_sentence[i][1]))
    # Step 5 - Offcourse, output the summarize texr
    summ_text=". ".join(summarize_text)   
    print("\n\nFor indexes of top ranked_sentence order press x and then enter:\n\n")
    return ranked_sentence,summ_text

In [92]:
x,y=generate_summary( "ab.txt",1)
print("Summarize Text: \n", ". ",y)



For indexes of top ranked_sentence order press x and then enter:


Summarize Text: 
 .  A VISUALLY STUNNING FILM TOPPED WITH GREAT PERFORMANCES STORY: A complex love story set during India’s Partition era, where the young Roop (Alia Bhatt) is torn between respect for husband Dev (Aditya Roy Kapur) and her newfound love for Zafar (Varun Dhawan). The other high points of the film are the problematic equation between father and son, Dev and Balraj as well as the fragile teacher-student dynamic between Bahaar Begum and Roop. While their back stories and heart-breaking love saga unfolds, the history of India takes an epic turn, from where there is no coming backREVIEW: As the name suggests, 'Kalank' is a story that questions the high morals of society, especially when it comes to love and family ties. In that aspect, writer and director Abhishek Varman's film makes a strong point.The story is based in a town called Husnabad, near Lahora few years before the Partition of India and Pakistan

In [95]:
x,y=generate_summary( "https://www.filmfare.com/reviews/bollywood-movies/movie-review-the-tashkent-files-33275.html",2)
print("Summarize Text: \n",y)

 The Tashkent Files Story: A young and ambitious journalist Ragini (Shweta Basu Prasad) receives an anonymous tip about a scoop behind India’s ex-Prime Minister Lal Bahadur Shastri’s mysterious death in Tashkent. She publishes a story in the newspaper, which leads to an official committee being set up by the government to investigate the 53-year-old case. The Tashkent Files Review: Based on one of the most controversial chapters of India’s history, The Tashkent Files tries to recreate a hard-hitting and volatile political drama. The film is based in modern times, as a journalist takes a deep dive into investigating Lal Bahadur Shastri’s death from 1966. While the pursuit of truth after half-a-century may seem jaded, the young journalist, played by Shweta Basu Prasad, chases the scoop because she’s overly ambitious and in the world of social media, any sensationalist topic, will do. So she and a whole host of politicians, social workers and civil servants, join a committee that has the 

In [82]:
string="In an attempt to build an AI-ready workforce, Microsoft announced Intelligent Cloud Hub which has been launched to empower the next generation of students with AI-ready skills. Envisioned as a three-year collaborative program, Intelligent Cloud Hub will support around 100 institutions with AI infrastructure, course content and curriculum, developer support, development tools and give students access to cloud and AI services. As part of the program, the Redmond giant which wants to expand its reach and is planning to build a strong developer ecosystem in India with the program will set up the core AI infrastructure and IoT Hub for the selected campuses. The company will provide AI development tools and Azure AI services such as Microsoft Cognitive Services, Bot Services and Azure Machine Learning.According to Manish Prakash, Country General Manager-PS, Health and Education, Microsoft India, said, ""With AI being the defining technology of our time, it is transforming lives and industry and the jobs of tomorrow will require a different skillset. This will require more collaborations and training and working with AI. That’s why it has become more critical than ever for educational institutions to integrate new cloud and AI technologies. The program is an attempt to ramp up the institutional set-up and build capabilities among the educators to educate the workforce of tomorrow." +"The program aims to build up the cognitive skills and in-depth understanding of developing intelligent cloud connected solutions for applications across industry. Earlier in April this year, the company announced Microsoft Professional Program In AI as a learning track open to the public. The program was developed to provide job ready skills to programmers who wanted to hone their skills in AI and data science with a series of online courses which featured hands-on labs and expert instructors as well. This program also included developer-focused AI school that provided a bunch of assets to help build AI skills."

# Reviewer vs public(opinion difference)

In [96]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer 
sid = SentimentIntensityAnalyzer()
ss = sid.polarity_scores(y)

In [97]:
ss

{'neg': 0.047, 'neu': 0.847, 'pos': 0.106, 'compound': 0.9802}

In [98]:
#general opinion of tashkent files critic is 